In [1]:
!pip install torch_geometric

In [2]:
import os
import math
import time
import shutil
import pickle
import numpy as np
import pandas as pd
import networkx as nx
import scipy.sparse as sp
from scipy.sparse.linalg import eigsh
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import Parameter
from torch_geometric.data import Data, InMemoryDataset, download_url
from torch_geometric.utils import to_scipy_sparse_matrix
from sklearn.metrics import accuracy_score, f1_score, average_precision_score, roc_auc_score, mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr
from sklearn.preprocessing import MinMaxScaler

# --- Helper Functions for Spectral GCN (Required by your model code) ---
import scipy.sparse as sp
from scipy.sparse.linalg import eigsh
import torch

def scaled_Laplacian(W):
    '''
    Compute \tilde{L} using Scipy Sparse Matrices to save RAM
    '''
    # W is expected to be a scipy.sparse matrix (CSR/COO)
    assert W.shape[0] == W.shape[1]
    
    # Tính Degree Matrix (D) dạng thưa
    D_values = np.array(W.sum(1)).flatten()
    D = sp.diags(D_values)
    
    L = D - W
    
    # Tính giá trị riêng lớn nhất (lambda_max) trên ma trận thưa
    # k=1: chỉ lấy 1 giá trị riêng lớn nhất
    lambda_max = eigsh(L, k=1, which='LM', return_eigenvectors=False)[0]
    
    # Công thức: (2 * L / lambda_max) - I
    L_tilde = (2 * L / lambda_max) - sp.eye(W.shape[0])
    return L_tilde

def cheb_polynomial(L_tilde, K):
    '''
    Compute Chebyshev polynomials but keep them as Sparse Tensors
    '''
    N = L_tilde.shape[0]
    
    # Chuyển đổi Scipy Sparse -> PyTorch Sparse Tensor
    def scipy_to_torch_sparse(probs):
        coo = probs.tocoo()
        values = coo.data
        indices = np.vstack((coo.row, coo.col))
        
        i = torch.LongTensor(indices)
        v = torch.FloatTensor(values)
        shape = coo.shape
        
        # Trả về sparse tensor
        return torch.sparse_coo_tensor(i, v, torch.Size(shape))

    L_tilde_torch = scipy_to_torch_sparse(L_tilde)
    
    # T0 = I, T1 = L_tilde
    indices_eye = torch.arange(N)
    indices_eye = torch.stack((indices_eye, indices_eye))
    values_eye = torch.ones(N)
    T0 = torch.sparse_coo_tensor(indices_eye, values_eye, torch.Size((N, N)))
    
    cheb_polynomials = [T0, L_tilde_torch]
    
    # Đệ quy: Tk = 2 * L * Tk-1 - Tk-2
    # Lưu ý: Nhân 2 ma trận sparse trong PyTorch đôi khi tốn kém, 
    # nhưng ở đây ta tính trước (pre-compute) nên chấp nhận được.
    for i in range(2, K):
        # Sparse MM: L_tilde (NxN) @ T_{i-1} (NxN)
        # PyTorch hỗ trợ sparse @ sparse -> sparse (trong các version mới)
        # Hoặc dùng torch.sparse.mm (cho sparse @ dense). 
        # Để an toàn và tiết kiệm RAM, ta giữ list dạng sparse tensor.
        
        # Với K nhỏ (2, 3), ta có thể xấp xỉ hoặc tính toán cẩn thận.
        # Để đơn giản và tránh lỗi version PyTorch cũ với sparse@sparse, 
        # ta sẽ thực hiện phép nhân này ngay trong model hoặc convert sang dense CHỈ KHI cần thiết.
        # Tuy nhiên, cách tốt nhất là giữ nguyên L_tilde và thực hiện phép nhân đệ quy trong forward pass
        # Nhưng để tuân thủ cấu trúc code cũ, ta sẽ lưu list này.
        
        # *Lưu ý*: PyTorch `spmm` (sparse x sparse) chưa ổn định ở mọi version.
        # Giải pháp tối ưu RAM cho người dùng: 
        # Chỉ lưu L_tilde và thực hiện phép tính Chebyshev "on-the-fly" hoặc 
        # chỉ dùng K=2 (thường đủ tốt).
        pass 
        
    # *FIX*: Để tránh lỗi tính toán phức tạp sparse-sparse, 
    # code này sẽ trả về list chứa các tham số để tính trong GCN layer
    # hoặc trả về list sparse tensor đã convert.
    # Ở đây tôi trả về list sparse tensor cho K=2 (phổ biến nhất).
    # Nếu K>2, cần dùng thư viện `torch_sparse` chuyên dụng, nhưng để đơn giản:
    return cheb_polynomials

In [3]:
class GCN(nn.Module):
    def __init__(self, L_tilde, dim_in, dim_out, order_K, device, in_drop=0.0, gcn_drop=0.0, residual=False):
        super(GCN, self).__init__()
        self.DEVICE = device
        self.order_K = order_K
        
        # L_tilde bây giờ là Sparse Tensor, không lưu dense
        self.L_tilde = L_tilde.to(device) 
        
        self.dim_in = dim_in
        self.dim_out = dim_out
        self.Theta = nn.ParameterList([nn.Parameter(torch.FloatTensor(dim_in, dim_out)) for _ in range(order_K)])
        self.weights = nn.Parameter(torch.FloatTensor(size=(dim_out, dim_out)))
        self.biases = nn.Parameter(torch.FloatTensor(size=(dim_out,)))
        self._in_drop = in_drop
        self._gcn_drop = gcn_drop
        self._residual = residual
        self.linear = nn.Linear(dim_in, dim_out)
        self.reset_parameters()

    def reset_parameters(self):
        for param in self.parameters():
            if param.dim() > 1:
                nn.init.xavier_uniform_(param)

    def forward(self, x, state=None, M=None):
        # x shape: (Batch, Nodes, Features) -> (B, N, F_in)
        batch_size = x.shape[0]
        num_nodes = x.shape[1]
        
        output = torch.zeros(batch_size, num_nodes, self.dim_out).to(self.DEVICE)
        
        # Chuẩn bị dữ liệu để nhân sparse: (N, N) sparse @ (N, Batch*F) dense
        # Reshape x: (B, N, F) -> (N, B, F) -> (N, B*F)
        x_reshaped = x.permute(1, 0, 2).reshape(num_nodes, -1)
        
        x0 = x
        if self._in_drop != 0:
            x = torch.dropout(x, 1.0 - self._in_drop, train=True)
            
        # Tính Chebyshev on-the-fly hoặc dùng pre-computed
        # Ở đây ta dùng đệ quy trực tiếp để tiết kiệm RAM lưu trữ các T_k
        
        # T_0(L) * x = I * x = x
        # Term 0
        support = x # (B, N, F)
        # support @ Theta_0
        out_k = torch.matmul(support, self.Theta[0])
        output = output + out_k
        
        if self.order_K > 1:
            # T_1(L) * x = L * x
            # Sparse MM: (N, N) @ (N, B*F) -> (N, B*F)
            Lx = torch.sparse.mm(self.L_tilde, x_reshaped)
            # Reshape lại về (B, N, F)
            Lx_out = Lx.reshape(num_nodes, batch_size, -1).permute(1, 0, 2)
            
            # Term 1
            out_k = torch.matmul(Lx_out, self.Theta[1])
            output = output + out_k
            
            # Lưu lại T_{k-1} và T_{k-2} để đệ quy nếu K > 2
            T_prev = Lx_out
            T_prev2 = x
            
            for k in range(2, self.order_K):
                # T_k = 2 * L * T_{k-1} - T_{k-2}
                # Tính L * T_{k-1}
                T_prev_reshaped = T_prev.permute(1, 0, 2).reshape(num_nodes, -1)
                L_Tprev = torch.sparse.mm(self.L_tilde, T_prev_reshaped)
                L_Tprev = L_Tprev.reshape(num_nodes, batch_size, -1).permute(1, 0, 2)
                
                T_k = 2 * L_Tprev - T_prev2
                
                # Term k
                out_k = torch.matmul(T_k, self.Theta[k])
                output = output + out_k
                
                # Update đệ quy
                T_prev2 = T_prev
                T_prev = T_k
            
        output = torch.matmul(output, self.weights)
        output = output + self.biases
        res = F.relu(output)
        
        if self._gcn_drop != 0.0:
            res = torch.dropout(res, 1.0 - self._gcn_drop, train=True)
        if self._residual:
            x0 = self.linear(x0)
            res = res + x0
        return res

class MGCN_Standard(nn.Module):
    def __init__(self, L_tilde, dim_in, dim_out, range_K, device, in_drop=0.0, gcn_drop=0.0, residual=False):
        super(MGCN_Standard, self).__init__()
        self.DEVICE = device
        self.K = range_K
        self.GCN_khops_node = nn.ModuleList([GCN(L_tilde, dim_in, dim_out, k + 1, device, in_drop=in_drop, gcn_drop=gcn_drop, residual=residual) for k in range(self.K)])
        self.linear = nn.Linear(dim_out, dim_in)
        self.W = nn.Parameter(torch.FloatTensor(dim_in, dim_out))
        self.b = nn.Parameter(torch.FloatTensor(dim_out, ))
        nn.init.xavier_uniform_(self.W)

    def forward(self, X):
        Xs = []
        for k in range(self.K):
            X_out = self.GCN_khops_node[k](X)
            # Attention mechanism part 1
            # X_out: (B, N, F_out)
            # linear maps back to dim_in? The code logic seems to try to combine inputs.
            # Let's follow the code strictly.
            # Note: In the provided code, self.linear maps dim_out -> dim_in.
            X_linear = self.linear(X_out) 
            X1 = torch.sigmoid(X_linear.matmul(self.W) + self.b)
            Xs.append(X1)
        Xs = torch.stack(Xs)
        return Xs

class MRA_GCN_Standard(nn.Module):
    def __init__(self, L_tilde, dim_in, dim_out, range_K, device, in_drop=0.0, gcn_drop=0.0, residual=False):
        super(MRA_GCN_Standard, self).__init__()
        self.DEVICE = device
        self.dim_out = dim_out
        self.W_a = nn.Parameter(torch.FloatTensor(self.dim_out, self.dim_out))
        self.U = nn.Parameter(torch.FloatTensor(self.dim_out))
        self.MGCN = MGCN_Standard(L_tilde, dim_in, dim_out, range_K, device, in_drop=in_drop, gcn_drop=gcn_drop, residual=residual)
        
        # Output layer for classification
        self.fc_final = nn.Linear(dim_out, 2) # Assuming binary classification for TRAVEL

        nn.init.xavier_uniform_(self.W_a)
        nn.init.uniform_(self.U)

    def forward(self, X):
        # X shape: (Batch, N, F) or (N, F)
        if X.dim() == 2:
            X = X.unsqueeze(0) # Add batch dimension if missing
            
        input = self.MGCN(X) # Shape: (K, B, N, F_out) - wait, MGCN returns stack of Xs
        
        # Attention pooling
        # input shape expected: (K, B, N, dim_out) based on MGCN return
        # Code: torch.einsum('ijkl,lm->ijkm', input, self.W_a)
        # Assuming input is (K, B, N, F)
        
        tmp = torch.einsum('kbnf,fm->kbnm', input, self.W_a)
        e = torch.einsum('kbnm,m->kbn', tmp, self.U)
        
        alpha = F.softmax(e, dim=0).unsqueeze(-1) # Softmax over K
        
        # Weighted sum
        h = torch.einsum('kbnf,kbnl->bnf', input, alpha) # (B, N, F)
        
        return h

In [4]:
# --- Dataset Loading from Task 1 ---
def read_npz(path):
    with np.load(path, allow_pickle=True) as f:
        return parse_npz(f)

def parse_npz(f):
    crash_time = f['crash_time']
    x = torch.from_numpy(f['x']).to(torch.float)
    coords = torch.from_numpy(f['coordinates']).to(torch.float)
    edge_attr = torch.from_numpy(f['edge_attr']).to(torch.float)
    cnt_labels = torch.from_numpy(f['cnt_labels']).to(torch.long)
    occur_labels = torch.from_numpy(f['occur_labels']).to(torch.long)
    # Some files might have different keys, handling basic ones
    edge_index = torch.from_numpy(f['edge_index']).to(torch.long).t().contiguous()
    return Data(x=x, y=occur_labels, edge_index=edge_index,
                edge_attr=edge_attr, coords=coords, cnt_labels=cnt_labels)

class TRAVELDataset(InMemoryDataset):
    url = 'https://github.com/baixianghuang/travel/raw/main/TAP-city/{}.npz'
    def __init__(self, root: str, name: str, transform=None, pre_transform=None):
        self.name = name.lower()
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0], weights_only=False)

    @property
    def raw_dir(self) -> str: return os.path.join(self.root, self.name, 'raw')
    @property
    def processed_dir(self) -> str: return os.path.join(self.root, self.name, 'processed')
    @property
    def raw_file_names(self) -> str: return f'{self.name}.npz'
    @property
    def processed_file_names(self) -> str: return 'data.pt'

    def download(self):
        download_url(self.url.format(self.name), self.raw_dir)

    def process(self):
        data = read_npz(self.raw_paths[0])
        data = data if self.pre_transform is None else self.pre_transform(data)
        data, slices = self.collate([data])
        torch.save((data, slices), self.processed_paths[0])

def train_test_split_stratify(dataset, train_ratio, val_ratio, class_num):
    labels = dataset[0].y
    train_mask = torch.zeros(size=labels.shape, dtype=bool)
    val_mask = torch.zeros(size=labels.shape, dtype=bool)
    test_mask = torch.zeros(size=labels.shape, dtype=bool)
    for i in range(class_num):
        stratify_idx = np.argwhere(labels.numpy() == i).flatten()
        np.random.shuffle(stratify_idx)
        split1 = int(len(stratify_idx) * train_ratio)
        split2 = split1 + int(len(stratify_idx) * val_ratio)
        train_mask[stratify_idx[:split1]] = True
        val_mask[stratify_idx[split1:split2]] = True
        test_mask[stratify_idx[split2:]] = True
    return train_mask, val_mask, test_mask

In [5]:
class MADGCN_Wrapper(nn.Module):
    def __init__(self, dataset, device, hidden_dim=64, K=2):
        super(MADGCN_Wrapper, self).__init__()
        self.device = device
        data = dataset[0]
        
        # --- FIX: Xử lý Sparse hoàn toàn ---
        print("Converting to Sparse Adjacency...")
        # 1. Lấy Edge Index
        edge_index = data.edge_index
        num_nodes = data.num_nodes
        
        # 2. Tạo Scipy Sparse Matrix (COO) trực tiếp, KHÔNG dùng toarray()
        # Tạo vector trọng số là 1 cho các cạnh
        values = np.ones(edge_index.shape[1])
        # Chuyển edge_index sang numpy
        indices = edge_index.cpu().numpy()
        
        adj_sparse = sp.coo_matrix((values, (indices[0], indices[1])), 
                                   shape=(num_nodes, num_nodes))
        
        # 3. Tính Laplacian dạng Sparse
        print("Computing Scaled Laplacian (Sparse)...")
        L_tilde_sparse = scaled_Laplacian(adj_sparse)
        
        # 4. Chuyển sang PyTorch Sparse Tensor
        def scipy_to_torch_sparse(probs):
            coo = probs.tocoo()
            values = coo.data
            indices = np.vstack((coo.row, coo.col))
            i = torch.LongTensor(indices)
            v = torch.FloatTensor(values)
            return torch.sparse_coo_tensor(i, v, torch.Size(coo.shape))

        self.L_tilde = scipy_to_torch_sparse(L_tilde_sparse).to(device)
        print("Laplacian computed and loaded to device.")
        
        # 5. Khởi tạo Model
        # Lưu ý: Ta truyền trực tiếp L_tilde sparse vào, và GCN class mới sẽ xử lý nó
        self.mra_gcn = MRA_GCN_Standard(
            L_tilde=self.L_tilde,
            dim_in=dataset.num_features,
            dim_out=hidden_dim,
            range_K=K,
            device=device,
            in_drop=0.5,
            gcn_drop=0.5
        )
        
        self.fc = nn.Linear(hidden_dim, dataset.num_classes)

    def forward(self, x):
        h = self.mra_gcn(x)
        if h.dim() == 3:
            h = h.squeeze(0)
        out = self.fc(h)
        return F.log_softmax(out, dim=1)
        
# --- Training & Evaluation Function ---
def train_eval_pipeline(city_name='Miami', state_name='Florida'):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Paths
    file_path = 'exp/'
    city_formatted = city_name.lower().replace(" ", "_") + '_' + state_name
    
    # Prepare Data
    if os.path.exists(file_path+city_formatted+'/processed'):
        shutil.rmtree(file_path+city_formatted+'/processed')
    
    print(f"Loading dataset for {city_name}...")
    dataset = TRAVELDataset(file_path, city_formatted)
    data = dataset[0]
    
    # Split & Normalize (Logic from Task 1)
    class_num = dataset.num_classes
    data.train_mask, data.val_mask, data.test_mask = train_test_split_stratify(
        dataset, 0.6, 0.2, class_num)
    
    sc = MinMaxScaler()
    data.x[data.train_mask] = torch.tensor(sc.fit_transform(data.x[data.train_mask]), dtype=torch.float)
    data.x[data.val_mask] = torch.tensor(sc.transform(data.x[data.val_mask]), dtype=torch.float)
    data.x[data.test_mask] = torch.tensor(sc.transform(data.x[data.test_mask]), dtype=torch.float)
    
    data = data.to(device)
    
    # Initialize Model (The MADGCN integration)
    model = MADGCN_Wrapper(dataset, device, hidden_dim=32, K=2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    
    # Training Loop
    epochs = 200
    best_val_f1 = 0
    test_metrics = {}
    
    print("Start Training MADGCN (MRA_GCN variant)...")
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        out = model(data.x)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        
        # Validation
        model.eval()
        with torch.no_grad():
            logits = model(data.x)
            pred = logits.max(1)[1]
            
            # Val metrics
            val_f1 = f1_score(data.y[data.val_mask].cpu(), pred[data.val_mask].cpu(), average='binary')
            
            if val_f1 > best_val_f1:
                best_val_f1 = val_f1
                
                # Calculate Test Metrics (MAE, RMSE, PCC)
                mask = data.test_mask
                y_true = data.y[mask].cpu().numpy()
                y_pred = pred[mask].cpu().numpy()
                y_prob = torch.exp(logits[mask])[:, 1].cpu().numpy() # Probability of class 1
                
                acc = accuracy_score(y_true, y_pred)
                auc = roc_auc_score(y_true, y_prob)
                
                # Regression metrics on binary labels (as requested)
                mae = mean_absolute_error(y_true, y_pred)
                rmse = np.sqrt(mean_squared_error(y_true, y_pred))
                pcc, _ = pearsonr(y_true, y_pred)
                
                test_metrics = {
                    'F1': val_f1,
                    'ACC': acc,
                    'AUC': auc,
                    'MAE': mae,
                    'RMSE': rmse,
                    'PCC': pcc
                }
        
        if epoch % 20 == 0:
            print(f'Epoch {epoch:03d} | Loss: {loss:.4f} | Val F1: {val_f1:.4f}')

    print("\nFinal Results for MADGCN:")
    print(f"City: {city_name}")
    print(f"Accuracy: {test_metrics['ACC']:.4f}")
    print(f"MAE: {test_metrics['MAE']:.4f}")
    print(f"RMSE: {test_metrics['RMSE']:.4f}")
    print(f"PCC: {test_metrics['PCC']:.4f}")
    
    return test_metrics

# --- Execute ---
# Chạy thử với Dallas, TX (thường có dữ liệu ổn định trong bộ TRAVEL)
# Bạn có thể đổi thành ('Miami', 'Florida') hoặc ('Los Angeles', 'California')
us_state_to_abbrev = {
    "Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR", "California": "CA", 
    "Colorado": "CO", "Connecticut": "CT", "Florida": "FL", "Georgia": "GA", "Texas": "TX",
    "New York": "NY" 
} # (Thêm đầy đủ nếu cần từ code cũ)

if __name__ == "__main__":
    results = train_eval_pipeline('Los Angeles', 'ca')

Using device: cuda
Loading dataset for Los Angeles...


Processing...
Done!


Converting to Sparse Adjacency...
Computing Scaled Laplacian (Sparse)...
Laplacian computed and loaded to device.
Start Training MADGCN (MRA_GCN variant)...
Epoch 000 | Loss: 0.6710 | Val F1: 0.0000
Epoch 020 | Loss: 0.3889 | Val F1: 0.0000
Epoch 040 | Loss: 0.3738 | Val F1: 0.0000
Epoch 060 | Loss: 0.3409 | Val F1: 0.0000
Epoch 080 | Loss: 0.3378 | Val F1: 0.0951
Epoch 100 | Loss: 0.3328 | Val F1: 0.2291
Epoch 120 | Loss: 0.3298 | Val F1: 0.2602
Epoch 140 | Loss: 0.3294 | Val F1: 0.2358
Epoch 160 | Loss: 0.3275 | Val F1: 0.2533
Epoch 180 | Loss: 0.3272 | Val F1: 0.2386

Final Results for MADGCN:
City: Los Angeles
Accuracy: 0.8853
MAE: 0.1147
RMSE: 0.3387
PCC: 0.3348
